In [1]:
from web3 import Web3
import json
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Read environment variables
rpc_url = os.getenv("RPC_URL")
owner_private_key = os.getenv("OWNER")
A_private_key = os.getenv("PARTY_A")
B_private_key = os.getenv("PARTY_B")

# Connect to the Ethereum network
# infura_url = "https://mainnet.infura.io/v3/YOUR_INFURA_PROJECT_ID"
web3 = Web3(Web3.HTTPProvider(rpc_url))

# Verify the connection
if not web3.is_connected():
    raise Exception("Failed to connect to Ethereum network")


In [2]:

# Function to send a transaction
def send_transaction(function_call,address,private_key):
    nonce = web3.eth.get_transaction_count(address)
    print(nonce)
    txn = function_call.build_transaction({
        'chainId': 84532,  # Mainnet chain ID
        'gas': 500000,
        'gasPrice': web3.to_wei('0.1', 'gwei'),
        'nonce': nonce,
    })
    signed_txn = web3.eth.account.sign_transaction(txn, private_key=private_key)
    tx_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
    tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    return tx_receipt

# Function to read a variable from the contract
def read_contract_variable(contract, variable_name):
    contract_function = getattr(contract.functions, variable_name)
    result = contract_function().call()
    return result

In [5]:
contract_abi = json.loads("""[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"address","name":"owner","type":"address"}],"name":"OwnableInvalidOwner","type":"error"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"OwnableUnauthorizedAccount","type":"error"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"party","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"MarginDeposited","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"party","type":"address"}],"name":"MarginWithdrawed","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"amountA","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amountB","type":"uint256"}],"name":"PartyALiquidated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"amountA","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amountB","type":"uint256"}],"name":"PartyBLiquidated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"tokenAPayment","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"tokenBPayment","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"lastPaymentTime","type":"uint256"}],"name":"PaymentMade","type":"event"},{"anonymous":false,"inputs":[],"name":"SwapEnded","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"tokenAInitMargin","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"tokenBInitMargin","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"tokenAMaintenanceMargin","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"tokenBMaintenanceMargin","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"marketMakerFeeBps","type":"uint256"}],"name":"SwapMarginAndFeeSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"partyA","type":"address"},{"indexed":true,"internalType":"address","name":"partyB","type":"address"}],"name":"SwapSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"startTime","type":"uint256"}],"name":"SwapStarted","type":"event"},{"inputs":[{"internalType":"uint256","name":"depositValue","type":"uint256"}],"name":"depositMarginTokenA","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"depositValue","type":"uint256"}],"name":"depositMarginTokenB","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"endSwap","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"lastPaymentTime","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"makePayment","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"marketMakerFeeBps","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"partyA","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"partyALiquidation","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"partyB","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"partyBLiquidation","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"paymentInterval","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"renounceOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_tokenAInitMargin","type":"uint256"},{"internalType":"uint256","name":"_tokenBInitMargin","type":"uint256"},{"internalType":"uint256","name":"_tokenAMaintenanceMargin","type":"uint256"},{"internalType":"uint256","name":"_tokenBMaintenanceMargin","type":"uint256"},{"internalType":"uint256","name":"_marketMakerFeeBps","type":"uint256"}],"name":"setMarginFee","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_partyA","type":"address"},{"internalType":"address","name":"_partyB","type":"address"},{"internalType":"address","name":"_tokenA","type":"address"},{"internalType":"address","name":"_tokenB","type":"address"},{"internalType":"uint256","name":"_tokenANotional","type":"uint256"},{"internalType":"uint256","name":"_tokenBNotional","type":"uint256"},{"internalType":"uint256","name":"_paymentInterval","type":"uint256"},{"internalType":"uint256","name":"_totalDuration","type":"uint256"}],"name":"setSwap","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"startSwap","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"startTime","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"swapStarted","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"tokenA","outputs":[{"internalType":"contract IERC20","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"tokenAInitMargin","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"tokenAMaintenanceMargin","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"tokenANotional","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"tokenB","outputs":[{"internalType":"contract IERC20","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"tokenBInitMargin","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"tokenBMaintenanceMargin","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"tokenBNotional","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalDuration","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"withdrawMarginTokenA","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"withdrawMarginTokenB","outputs":[],"stateMutability":"nonpayable","type":"function"}]""")
contract_address = "0xd086BC6a9Bf19E066d6cBD07c27048e164ff3e94"

# Create the contract instance
contract = web3.eth.contract(address=contract_address, abi=contract_abi)

In [6]:
# Set up account
private_key = A_private_key
account = web3.eth.account.from_key(private_key)
address = account.address
address


'0xf52926F4D0EB85FF878d9ddC2577abc429addC9d'

In [8]:

func_call = contract.functions.depositMarginTokenA(100*10**18) # By default, ERC20 uses a value of 18 for decimals
receipt = send_transaction(func_call,address,private_key)


16
